In [ ]:
AWS production is integrated with Splunk ES using Splunk Add-on for AWS.
Logs are stored in aws_security index and collected using two best-practice methods:

Pull-based Modular Inputs + Push-based HEC inputs.

Pull-based: Splunk goes to the source and collects logs (Splunk initiates the connection)

Push-based: Source sends logs to Splunk (Source initiates the connection)

| Application/Device name | Source Types               | Data Collection Method | Data Collection Frequency | Index Name   | Platform | Syslog Servers              |
| ----------------------- | -------------------------- | ---------------------- | ------------------------- | ------------ | -------- | --------------------------- |
| AWS                     | aws:cloudtrail             | Modular Input          | Every 5 minutes           | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |
| AWS                     | aws:cloudwatch             | Modular Input          | Every 5 minutes           | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |
| AWS                     | aws:config                 | Modular Input          | Every 3 minutes           | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |
| AWS                     | aws:s3:accesslogs          | Modular Input          | Every 5 minutes           | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |
| AWS                     | aws:cloudwatch:guardduty   | Modular Input          | Every 30 seconds          | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |
| AWS                     | aws:config:rule            | Modular Input          | Every 5 minutes           | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |
| AWS                     | aws:metadata               | Modular Input          | Every 10 minutes          | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |
| AWS                     | aws:securityhub:finding    | HEC Input              | Real Time                 | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |
| AWS                     | aws:cloudwatchlogs:vpcflow | HEC Input              | Real Time                 | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |
| AWS                     | aws:accessanalyzer:finding | HEC Input              | Real Time                 | aws_security | AWS Prod | SPLUNK-HFSYS02 / 10.2.8.142 |


Configuration On Deployment Server: 
A Serverclass afrex_all_aws_addon is created on the Deployment server for deploying the below add-on 
on the AWS Syslog/HF02 server to collect various AWS log sources.

Update serverclass.conf: 
To avoid overwriting/deleting inputs during add-on upgrades, edit the serverclass.conf file on Deployment 
server and add the below parameter under the Salesforce serverclass as shown below 
excludeFromUpdate = $app_root$/local 
/opt/splunk/etc/system/local/serverclass.conf 
[serverClass:afrexim_all_aws_addon:app:Splunk_TA_aws] 
restartSplunkWeb = 0 
restartSplunkd = 1 
stateOnClient = enabled 
excludeFromUpdate = $app_root$/local

Configure AWS API Account on Splunk 
Collect Key ID, Secret Key of an AWS API account to integrate splunk with AWS cloud and add the account 
details on AWS Syslog/HF02 server as shown below.

More Info: 
https://www.cloud.northwestern.edu/resources/howtos/sending-aws-logs-to-splunk/ 
https://docs.splunk.com/Documentation/AddOns/released/AWS/Setuptheadd-on 


Short & simple:

CloudTrail → tracks who did what in AWS (API activity/audit)

CloudWatch → collects AWS service/system logs & metrics

Config / Config Rules → tracks resource changes & compliance

GuardDuty → collects threat detections

S3 Access Logs → tracks who accessed S3 buckets/objects

Metadata → collects AWS account/resource inventory details


CloudWatch Log Group generates logs from AWS services/applications.

We attach a Subscription Filter to that log group, which forwards every new log event in real time to Lambda.

Lambda receives the CloudWatch log batch (awslogs.data), decodes/decompresses it, and stores the processed logs as .json.gz files in S3.

On every new file creation, S3 Event Notification triggers SNS to publish a message.

SNS forwards that message to SQS, so messages are buffered safely.

Splunk TA-AWS pulls SQS continuously, reads the message, and gets the S3 object path.

Splunk then downloads the file from S3 and indexes the events into the aws_security index.

AWS Admin tasks

Create S3 bucket + prefix

Create IAM policy + IAM role for Lambda

Create Lambda function (CloudWatch logs → write to S3)

Create/identify CloudWatch Log Group

Create Subscription Filter (Log Group → Lambda)

Create SNS topic

Create SQS queue

Subscribe SQS to SNS

Configure SNS policy (allow S3 to publish)

Configure S3 Event Notification (S3 → SNS)

Create IAM user/role for Splunk + provide Access/Secret keys + Queue URL

✅ Splunk Admin tasks

Install/configure Splunk Add-on for AWS (TA-AWS)

Add AWS Account in TA-AWS (Access Key + Secret + Region)

Create SQS-Based S3 input

Queue URL

S3 bucket/prefix/suffix

Index = aws_security

Sourcetype

Validate ingestion in Splunk using search and monitoring logs

Queue URL ?
✅ What is Queue URL?

Queue URL = the unique web address of the SQS queue that Splunk uses to read messages.

It looks like this:
https://sqs.<region>.amazonaws.com/<account-id>/<queue-name>

Example:
https://sqs.ap-south-1.amazonaws.com/123456789012/cloudwatch-splunk-queue




SQS Queue URL (must, not ARN)

S3 Bucket name

S3 Prefix (folder path)

S3 Suffix (example .json.gz)

Region (ap-south-1 etc.)

IAM Access key/Secret OR IAM Role assumption for Splunk

Confirm SNS → SQS subscription is done

Confirm CloudWatch log group name

CloudWatch subscription filter → Lambda → S3 and S3 event → SNS → SQS are configured.


1) Check if logs are coming (basic)
index=aws_security earliest=-15m

2) Check CloudWatch sourcetype only
index=aws_security sourcetype="aws:cloudwatch:logs" earliest=-15m

3) Check latest event time (data freshness)
index=aws_security sourcetype="aws:cloudwatch:logs"
| stats latest(_time) as last_event_time count
| convert ctime(last_event_time)

4) Check if there is a gap (missing logs)
index=aws_security sourcetype="aws:cloudwatch:logs" earliest=-2h
| timechart span=5m count


✅ If some 5-min buckets show 0, ingestion delay/issues.

5) Check TA-AWS errors in Splunk (important)
index=_internal sourcetype=splunkd ("aws" OR "SQS" OR "S3") ("ERROR" OR "AccessDenied" OR "Signature")

6) Confirm SQS polling working
index=_internal sourcetype=splunkd "sqs" ("ReceiveMessage" OR "poll")

7) Confirm S3 download happening
index=_internal sourcetype=splunkd "s3" ("GetObject" OR "download")
